#### Numpy EKF 

Here, we present a verion of the EKF that only utilies the numpy library. This avoids the dabocle that came with the sympy library.

##### The Model 

The system will use the following measurement model: 

> $z = \begin{pmatrix} z_I \\ z_Q \end{pmatrix} = \begin{pmatrix} a*cos(\theta) \\ a*sin(\theta) \end{pmatrix} + \nu_k$

Where $a$ is the signal amplitude, $z_I$ and $z_Q$ represent the in phase and quadrature sample of the incoming signal, repectively. 

The estimation of the signal amplitude will require a state augmentation. We will assume amplitude constancy as a dynamic model.

After linearization, the measurement model will be: 

> $z-z(a^*,\theta^*) = \begin{bmatrix} -a*sin(\theta^*) & 0 & 0 &  cos(\theta^*) \\ a*cos(\theta^*) & 0 & 0 & sin(\theta^*)\end{bmatrix} (x-x^*) + \nu $
>> == $z_{k}^{*}+H_{k}^{*}x^*= H_{k}^{*}x +\nu_k$

where all $(.)^*$ values come from some nominal trajectory and the state-space system becomes:

> $\begin{pmatrix} \dot{\theta} \\ \dot{v} \\ \dot{b} \\ \dot{a} \end{pmatrix} = \begin{bmatrix} 0 & \frac{1}{v_d + v_s} & 0 & 0 \\ 0 & -2\Omega^{2}_{i/e} & 1 & 0 \\ 0 & 0 & \frac{1}{\tau} & 0 \\ 0 & 0 & 0 & 0 \end{bmatrix} \begin{pmatrix} \theta \\ v \\ b \\ a \end{pmatrix} + \begin{bmatrix} 0 & 0 & 1 \\ 1 & 1 & 0 \\ 0 & 0 & 0 \\ 0 & 0 & 0  \end{bmatrix} \begin{pmatrix} f \\ g \\ \frac{v_d}{v_d+v_s} \end{pmatrix} + \begin{bmatrix} 1 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 1 \\ 0 & 0 & 0 \end{bmatrix} \begin{pmatrix} \omega_{LO} \\ \omega_{v} \\ \omega_{IMU} \end{pmatrix}$ 

In [2]:
# import numpy as np
import autograd.numpy as np
from autograd import jacobian

In [3]:
# system's constant parameters 
v_d = 343                                       # speed of sound in dry air in m/s
v_0 = 5                                         # initial magnitude of velocity 
f_d0 = 10                                       # initial doppler frequency in Hz
v_s = (v_d+v_0)/f_d0 - v_d                      # solving for initial satellite velocity in m/s. This will be estimated later.
omega_e = 1.99*(10**-7)                         # angular velocity of earth in rad/s 
tau = 1                                         # time constant of accelerometer bias 
phi_0 = np.pi/2                                 # initial phase offset of incoming signal
a_0 = 1                                         # initial signal amplitude

# simulation settings 
Ts = 10                                         # sample time in Hz
Tc = .02                                        # coherent integration time in seconds 
st = 60                                         # requested simulation runtime in seconds 

#### Non-linear dynamic adjustment 

Take the following system: 

> $x_k = F(x_{k-1}, u_{k-1}, w_{k-1})$

Linearizing 

> $x_k \approx x_{k}(x^*_{k-1}, u^*_{k-1}, w^*_{k-1}) + \frac{dF}{dx}|_{x = x^*}(x_{k-1} - x^*_{k-1}) + \frac{dF}{du}|_{u = u^*_{k-1}} (u_{k-1} - u^*_{k-1}) + \frac{dF}{dw}|_{w = w^*} (w_{k-1} - w^*_{k-1})$

This can be re-arranged into 

>$ x_k = \frac{dF}{dx}|_{x = x^*} x_{k-1} + F_{adj} \begin{pmatrix}  x_{k}(x^*_{k-1}, u^*_{k-1}, w^*_{k-1}) & x^*_{k-1} & u_{k-1} & u^*_{k-1} & w^*_{k-1} \end{pmatrix}^T + \frac{dF}{dw}|_{w = w^*} w_{k-1}$

Where 

> $F_{adj} = \begin{bmatrix} I & -\frac{dF}{dx}|_{x = x^*} & F' & -\frac{dF}{du}|_{u = u^*_{k-1}} & -\frac{dF}{dw}|_{w = w^*}\end{bmatrix}$
>> $EX: F_{adj} = x_{k}(x^*_{k-1}, u^*_{k-1}, w^*_{k-1}) -\frac{dF}{dx}|_{x = x^*} x^*_{k-1} + F'u_{k-1} - \frac{dF}{du}|_{u = u^*_{k-1}}u^*_{k-1} -\frac{dF}{dw}|_{w = w^*}w^*_{k-1}$

Where

>$F' = \left\{ \begin{array}{lr} \frac{dF}{du}|_{u = u^*_{k-1}}, & \text{when} \ \mathbb{J} \neq 0\\ F,&  \text{otherwise} \end{array} \right\}$
>> NOTE $w^* = E[w] = 0$

> Take $x_{k}(x^*_{k-1}, u^*_{k-1}, w^*_{k-1}) = x^*_k$
>> $x_k \approx  \frac{dF}{dx}|_{x = x^*} x_{k-1} + \begin{bmatrix} I & -\frac{dF}{dx}|_{x = x^*} & F' & -\frac{dF}{du}|_{u = u^*_{k-1}} \end{bmatrix}  \begin{pmatrix}  x^*_{k} & x^*_{k-1} & u_{k-1} & u^*_{k-1}\end{pmatrix}^T + \frac{dF}{dw}|_{w = w^*} w_{k-1}$ Note that $u_k = u^*_k$ in nearly all cases since this is a deterministic input.

Finally, we introduce the following convention 

> $x_k \approx F^*_x x_{k-1} + F_{adj} \bar{u}_{k-1} + F^*_w w_{k-1}$
>> $F^*_x = \frac{dF}{dx}|_{x = x^*} \\ F_{adj} = \begin{bmatrix} I & -\frac{dF}{dx}|_{x = x^*} & F' & -\frac{dF}{du}|_{u = u^*_{k-1}} \end{bmatrix} \\ \bar{u} = \begin{pmatrix}  x^*_{k} & x^*_{k-1} & u_{k-1} & u^*_{k-1}\end{pmatrix}^T \\ F^*_w = \frac{dF}{dw}|_{w = w^*}$  

In [ ]:
def ekf_update(x_hat, x, u, w, z, Phat, F_x, F_u, F_w, H_func, Q, R, jac_u): # TODO: don't pass in w like this. Generate it. 
    """
    Implements the extended Kalman filter algorithm.
    # TODO show new function args 
    Args:
        xhat (ndarray): priori for states in this epoch.
        x (ndarray): State vector to linearize off of.
        z (ndarray): The measurement vector.
        u (ndarray): The control input vector to linearize off of.
        w (ndarray): The noise at this time step.
        Phat (ndarray): Priori covariance of xhat.
        F_func (callable): The state transition function F(x,u).
        H_func (callable): The measurement function H(x).
        Q_func (callable): The process noise covariance function Q(x,u).
        R_func (callable): The measurement noise covariance function R(x).
        jac_u (ndarray): The jacobian of F_func wrt u. Used in Fadj.

    Returns:
        tuple: A tuple containing the updated state vector and state covariance matrix.
    """
        
    # Time update
    xbar = F_x*xhat + F_u*u                     # Calculate the predicted state vector. 
    Pbar = F_x @ Phat @ F_x.T + F_w @ Q @ F_w.T  # This is our "P_bar"      
    
    # Measurement Update
    H_x = is_linear(H_func, x) # Check if H_func is linear, and compute the Jacobian if isn't 
    S = H_x @ Pbar @ H_x.T + R 
    K = Pbar @ H_x.T @ np.linalg.inv(S)        # calculate kalman gain
    xhat = xbar + K @ (z - H_x @ xbar)      
    Phat = (np.eye(len(x)) - K @ H_x) @ Pbar 

    return xhat, Phat

def is_linear(func, argnumber, *args):
    """Check if a function is linear or nonlinear using autograd. If it is, apply the points to linearize off of."""
    jac = jacobian(func, argnum = argnumber)(*args)
    # if the jacobian is approximately zero, there's no need to linearize the function. 
    linear = np.allclose(jac, np.zeros_like(jac)) 
    if linear: 
        return func
    else: 
        return jac

In [ ]:
#TODO define F_func (non-linear dyanamics) and trajectory for x,y,z    

# this let's us return the jacobian of a matrix as a lambda function, so that these all only need to be evaluated once. 
def get_jacobian_func(func, argnum):
    jacobian_func = jacobian(func, argnum=argnum)
    return lambda *args: jacobian_func(*args)

def F_func(x, u):
    return np.array([x**2, np.sin(u)])

# returns lamba function forms of jacobians 
# TODO having one function in this way doesn't make sense in the case of a linear function 
F_x = get_jacobian_func(F_func, 0)  
F_u = get_jacobian_func(F_func, 1)
F_w = get_jacobian_func(F_func, 2)

# TODO define state trajectories and run EKF 
# Fadj explained above
Fadj = np.hstack(np.eye(F_x.shape), F_x)
Fadj = np.hstack(Fadj, )